In [ ]:
#!/usr/bin/env python3
# import sys
# sys.path.insert(0, r'PATH_TO_REPO')

import pandas as pd
import matplotlib.pyplot as plt

from src.utils.utils import get_files_in_from_directory

In [ ]:
all =   r'data\features.csv'
npm =   r'data\features_npm.csv'
pypi =  r'data\features_pypi.csv'
mvn =   r'data\features_mvn.csv'

df = pd.read_csv(npm)

In [ ]:
print(df.shape)
print((df['label_security_related']==True).sum())

In [ ]:
# Sanitize time_to_next_merge where nan is a valid value
max_merge = df['time_to_next_merge'].max() * 10
df.loc[df['time_to_next_merge'].isna(),'time_to_next_merge'] = max_merge

df.fillna(0, inplace=True)
df.dropna(axis=1, inplace=True)


In [ ]:
asd = {col:0 for col in df}

for col in df:
    nan_counts = df[col].isna().sum()
    asd[col] = nan_counts/df.shape[0]

asd = {k: v for k,v in sorted(asd.items(), key=lambda kv: -kv[1])}
# print(asd)


In [ ]:
import matplotlib.pyplot as plt

plt.violinplot(X['attack_in_file_content'])
print(X.shape)
print(X[X['attack_in_file_content'] > 0.8].shape)

In [ ]:
# added_lines_ratio_avg
# dmm_unit_size

# columns = X.columns
columns = ['']

df_sec = df[df['label_security_related'] == True]
for col in columns:
    dfx = df[df[col]<10] 
    dfx_sec = df_sec[df_sec[col]<10] 
    print(col)
    violin_parts = plt.violinplot(dfx[col])
    violin_parts['bodies'][0].set_facecolor('blue')
    violin_parts = plt.violinplot(dfx_sec[col])
    violin_parts['bodies'][0].set_facecolor('red')
    plt.ylim(0, 5)
    plt.show()

In [ ]:
%matplotlib qt   
import dateutil.parser as parser
from datetime import datetime, tzinfo
import pytz
#ether/etherpad-lite
#openpgpjs/openpgpjs


#added_lines_count_avg - good
#authent_in_title - bad
# to investigate: max_method_complexity_avg

repo = None #'ether/etherpad-lite'
if repo != None:
    dfx = df[df['label_repo_full_name']==repo]
else:
    dfx=df

print(dfx.shape)
df_sec = dfx[dfx['label_security_related'] == True]
x_axis = dfx['label_commit_date'].apply(parser.parse)
x_axis = [(x.replace(tzinfo=pytz.utc)-datetime.utcnow().replace(tzinfo=pytz.utc)).total_seconds() / 3600 for x in x_axis]
x_axis_sec = df_sec['label_commit_date'].apply(parser.parse)
x_axis_sec = [(x.replace(tzinfo=pytz.utc)-datetime.utcnow().replace(tzinfo=pytz.utc)).total_seconds() / 3600 for x in x_axis_sec]

columns = X
columns = ['authent_in_title']

for col in columns:
    # plt.title(col)
    plt.scatter(x_axis, dfx[col], color='b', marker='o')
    plt.xlim(-45000,10)
    plt.scatter(x_axis_sec, df_sec[col], color='r', marker='x')
    plt.ylabel('Commit contains the substring "authent" in the title')
    plt.xlabel('Time')
    plt.show()
    print(repo)

In [ ]:
broken_features = [
    'commits_to_next_merge',
    'commits_since_last_merge',
    'commits_to_next_merge'   
    'commits_since_last_merge'
]

highly_correlated_features = [
    'dmm_unit_complexity',
    'secur_in_title',
    'vulnerab_in_title',
    'exploit_in_title',
    'certificat_in_title',
    'authent_in_title',
    'leak_in_title',
    'sensit_in_title',
    'crash_in_title',
    'attack_in_title',
    'deadlock_in_title',
    'segfault_in_title',
    'malicious_in_title',
    'corrupt_in_title',
    'test_in_filename',
    'removed_lines_count_avg',
    'added_lines_count_avg',
    'changed_lines_count_avg',
    'modified_lines_count_avg',
    'file_size_avg',
    'changed_methods_count_max',
    'total_methods_count_avg',
    'max_method_token_count_avg'
    'avg_method_complexity_avg',
    'avg_method_complexity_max',
    'avg_method_parameter_count_max'

    'changes_to_file_in_prev_50_commits_avg',
    'changes_to_file_in_next_50_commits_avg'
]


df_new = df
df_new = df_new[df_new.columns.difference(broken_features)]
df_new = df_new[df_new.columns.difference(highly_correlated_features)]

X_new = df_new[df_new.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
y = df_new['label_security_related']

correlation_matrix = X_new.corr()
corr = correlation_matrix.values
column_names = correlation_matrix.columns

for i in range(len(column_names)):
    for j in range(i+1, len(column_names)):
        if abs(corr[i,j])> 0.5:
            print(column_names[i], ' ', column_names[j], ' ', corr[i,j])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_new, y)
print(X_new.shape)
print(X_rus.shape)

VERBOSE_LVL = 10
n_iter = 15

param_dist = {
    'n_estimators': list([50, 75, 100])
    }

model = RandomForestClassifier(random_state=42)
model_random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=n_iter,
    scoring = 'f1',
    refit=True,
    cv=5,
    verbose=VERBOSE_LVL)

result = model_random_search.fit(X_rus, y_rus)
selector = RFECV(result.best_estimator_, prefit=True)

X_new2 = selector.transform(X_new)
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_new2, y)
result = model_random_search.fit(X_rus, y_rus)

# result = model_random_search.fit(X_new_new, y)

